<a href="https://colab.research.google.com/github/Deepaksai3574/Scheduling-algorithms/blob/main/SIP_Journal_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'alzheimers-dataset-4-class-of-images:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F457093%2F861496%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240422%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240422T151923Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D9a38cf58f340901568d44361abd5594575eeb4b962f9a2ea86739be2f09f2f7054c9314f9083f809c654371d36dccfc4adc2fdc16d2feec41c1aa4803a1277c23eb97d1ab1e533973a25847a2c5460fcc1f2e7e08102f46e02405851cfb2a991e6e9e1b4d61f0ac999da40ad041a653cae31090aa366e2b5333c8bd30f88351d89cc52fffeed4b48b015d65a3c2ff4508706683265c01e72479367f287fb6916a52728da8bc79cfd8ed2695f0a0d6c36f579c49fc7e6cc3dc2b05db2db131e147cbb8518acb3f6be30b9a93a2f2d6544bb7b5e84829f2b0f2f88753ac4ee1d9a4f8c693463a31bfe62bb926c210cf52ae8383d23a4b85f8d6349473bf95b3b92'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import confusion_matrix, classification_report
from tqdm import tqdm
from imblearn.over_sampling import SMOTE
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.applications import ResNet50, ResNet152, EfficientNetB0, EfficientNetB1, EfficientNetB2
from keras import optimizers
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
images = []
labels = []
for subfolder in tqdm(os.listdir('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset')):
    subfolder_path = os.path.join('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset', subfolder)
    for folder in os.listdir(subfolder_path):
        subfolder_path2=os.path.join(subfolder_path,folder)
        for image_filename in os.listdir(subfolder_path2):
            image_path = os.path.join(subfolder_path2, image_filename)
            images.append(image_path)
            labels.append(folder)
df = pd.DataFrame({'image': images, 'label': labels})
df

In [ ]:
plt.figure(figsize=(10, 10))
labels = df['label'].value_counts().index
sizes = df['label'].value_counts().values
colors = sns.color_palette('Set1')

plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=140)
plt.title('Distribution of Classes', fontsize=20)
plt.show()

In [ ]:
plt.figure(figsize=(15,8))
ax = sns.countplot(x=df.label,palette='Set1')
ax.set_xlabel("Class",fontsize=20)
ax.set_ylabel("Count",fontsize=20)
plt.title('The Number Of Samples For Each Class',fontsize=20)
plt.grid(True)
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(50,50))
for n,i in enumerate(np.random.randint(0,len(df),50)):
    plt.subplot(10,5,n+1)
    img=cv2.imread(df.image[i])
    img=cv2.resize(img,(224,224))
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.axis('off')
    plt.title(df.label[i],fontsize=25)

In [ ]:
Size=(176,176)
work_dr = ImageDataGenerator(
    rescale = 1./255
)
train_data_gen = work_dr.flow_from_dataframe(df,x_col='image',y_col='label', target_size=Size, batch_size=6500, shuffle=False)

In [ ]:
train_data, train_labels = train_data_gen.next()

In [ ]:
class_num=np.sort(['MildDemented','ModerateDemented','NonDemented','VeryMildDemented'])
class_num

In [ ]:
sm = SMOTE(random_state=42)
train_data, train_labels = sm.fit_resample(train_data.reshape(-1, 176 * 176 * 3), train_labels)
train_data = train_data.reshape(-1, 176,176, 3)
print(train_data.shape, train_labels.shape)

In [ ]:
labels=[class_num[i] for i in np.argmax(train_labels,axis=1) ]
plt.figure(figsize=(15,8))
ax = sns.countplot(x=labels,palette='Set1')
ax.set_xlabel("Class",fontsize=20)
ax.set_ylabel("Count",fontsize=20)
plt.title('The Number Of Samples For Each Class',fontsize=20)
plt.grid(True)
plt.xticks(rotation=45)
plt.show()

In [ ]:
labels = [class_num[i] for i in np.argmax(train_labels, axis=1)]

plt.figure(figsize=(10, 10))
unique_labels = np.unique(labels)
sizes = [labels.count(label) for label in unique_labels]
colors = sns.color_palette('Set1')

plt.pie(sizes, labels=unique_labels, colors=colors, autopct='%1.1f%%', startangle=140)
plt.title('Distribution of Classes', fontsize=20)
plt.show()

In [ ]:
X_train, X_test1, y_train, y_test1 = train_test_split(train_data,train_labels, test_size=0.3, random_state=42,shuffle=True,stratify=train_labels)
X_val, X_test, y_val, y_test = train_test_split(X_test1,y_test1, test_size=0.5, random_state=42,shuffle=True,stratify=y_test1)
print('X_train shape is ' , X_train.shape)
print('X_test shape is ' , X_test.shape)
print('X_val shape is ' , X_val.shape)
print('y_train shape is ' , y_train.shape)
print('y_test shape is ' , y_test.shape)
print('y_val shape is ' , y_val.shape)

In [ ]:
cnn_model = Sequential()
cnn_model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(176, 176, 3)))
cnn_model.add(MaxPooling2D((2, 2)))
cnn_model.add(Conv2D(64, (3, 3), activation='relu'))
cnn_model.add(MaxPooling2D((2, 2)))
cnn_model.add(Conv2D(128, (3, 3), activation='relu'))
cnn_model.add(MaxPooling2D((2, 2)))
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation='relu'))
cnn_model.add(Dropout(0.5))
cnn_model.add(Dense(4, activation='softmax'))

# Compile the CNN model
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the CNN model
cnn_history = cnn_model.fit(
    X_train, y_train,
    epochs=10,
    validation_data=(X_val, y_val),
    verbose=1
)

# Save the CNN model
cnn_model.save('/kaggle/working/cnn_model.h5')

In [ ]:
resnet50_base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(176, 176, 3))
resnet50_model = Sequential()
resnet50_model.add(resnet50_base_model)
resnet50_model.add(Flatten())
resnet50_model.add(Dense(256, activation='relu'))
resnet50_model.add(Dropout(0.5))
resnet50_model.add(Dense(4, activation='softmax'))
resnet50_model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizers.RMSprop(learning_rate=1e-4),
    metrics=['accuracy']
)

# Train ResNet50 model
resnet50_history = resnet50_model.fit(
    X_train, y_train,
    epochs=10,
    validation_data=(X_val, y_val),
    verbose=1
)

# Save ResNet50 model
resnet50_model.save('/kaggle/working/resnet50_model.h5')

In [ ]:
!pip install EfficientNet

In [ ]:
from keras.applications import ResNet152
from efficientnet.keras import EfficientNetB2

# ... (Your existing code)

# Define ResNet152 model
resnet152_base_model = ResNet152(weights='imagenet', include_top=False, input_shape=(176, 176, 3))

# Freeze the initial 5 layers
for layer in resnet152_base_model.layers[:5]:
    layer.trainable = False

resnet152_model = Sequential()
resnet152_model.add(resnet152_base_model)
resnet152_model.add(Flatten())
resnet152_model.add(Dense(256, activation='relu'))
resnet152_model.add(Dropout(0.5))
resnet152_model.add(Dense(4, activation='softmax'))

resnet152_model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizers.RMSprop(learning_rate=1e-4),
    metrics=['accuracy']
)

resnet152_history = resnet152_model.fit(
    X_train, y_train,
    epochs=10,
    validation_data=(X_val, y_val),
    verbose=1
)

# Save ResNet152 model
resnet152_model.save('/kaggle/working/resnet152_model.h5')

In [ ]:
efficientnet_b2_model = Sequential()
efficientnet_b2_model.add(EfficientNetB2(weights='imagenet', include_top=False, input_shape=(176, 176, 3)))
efficientnet_b2_model.add(Flatten())
efficientnet_b2_model.add(Dense(256, activation='relu'))
efficientnet_b2_model.add(Dropout(0.5))
efficientnet_b2_model.add(Dense(4, activation='softmax'))

efficientnet_b2_model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizers.RMSprop(learning_rate=1e-4),
    metrics=['accuracy']
)

efficientnet_b2_history = efficientnet_b2_model.fit(
    X_train, y_train,
    epochs=10,
    validation_data=(X_val, y_val),
    verbose=1
)

# Save EfficientNetB2 model
efficientnet_b2_model.save('/kaggle/working/efficientnet_b2_model.h5')

In [ ]:
# ... (your existing code)

# Plot confusion matrices
def plot_confusion_matrix(model_name, y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=class_num, yticklabels=class_num)
    plt.title(f'Confusion Matrix - {model_name} Model')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

# Get predictions for each model on test data
y_pred_cnn_test = cnn_model.predict(X_test).argmax(axis=1)
y_pred_resnet50_test = resnet50_model.predict(X_test).argmax(axis=1)
y_pred_resnet152_test = resnet152_model.predict(X_test).argmax(axis=1)
y_pred_efficientnet_b2_test = efficientnet_b2_model.predict(X_test).argmax(axis=1)

# Plot confusion matrices for test data
plot_confusion_matrix('CNN', np.argmax(y_test, axis=1), y_pred_cnn_test)
plot_confusion_matrix('ResNet50', np.argmax(y_test, axis=1), y_pred_resnet50_test)
plot_confusion_matrix('ResNet152', np.argmax(y_test, axis=1), y_pred_resnet152_test)
plot_confusion_matrix('EfficientNetB2', np.argmax(y_test, axis=1), y_pred_efficientnet_b2_test)

# Plot training history (loss and accuracy) for each model
def plot_training_history(model_name, history):
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'{model_name} Model Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'{model_name} Model Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

# Plot training history for each model
plot_training_history('CNN', cnn_history)
plot_training_history('ResNet50', resnet50_history)
plot_training_history('ResNet152', resnet152_history)
plot_training_history('EfficientNetB2', efficientnet_b2_history)
